<h1><center>MERGE ACS DATA WITH FDNY DATA BY GEOID</center></h1>

In [210]:
import glob
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join

years = [str(i) for i in range(2010, 2019)]
path_from = 'C:\\Users\\mskac\\machineLearning\\GasLeakConEd\\data\\processed\\acs_data\\'
path_to = 'C:\\Users\\mskac\\machineLearning\\GasLeakConEd\\data\\processed\\merged_acs_data\\'
base_path = 'C:/Users/mskac/machineLearning/GasLeakConEd/data/'

files = ['acs_merged_data' + str(year) + '.csv' for year in range(2010, 2019)]

In [211]:
dfs_acs_original = [pd.read_csv(join(path_to, files[i])) for i in range(9)]
for i in range(len(dfs_acs_original)):
    dfs_acs_original[i]['incident_year'] = 2010 +i

In [212]:
# we will have all years information in one dataframe to merge it with gas info
df_acs = pd.concat(dfs_acs_original, ignore_index = True)
df_acs['geoid'] = df_acs['geoid'].astype('int64')

In [213]:
df_gas = pd.read_csv(base_path+ 'processed\FULL_fdny_2013_2020.csv')
df_gas['geoid'] = df_gas['geoid'].astype('int64')

In [214]:
df_gas_ = df_gas
df_gas_['incident_year'] = df_gas['incident_date_time'].str[6:10]
df_gas_['count'] = 1
df_gas_.head()

,Unnamed: 0,geoid,fire_box,ntaname,boroname,incident_date_time,engines_assigned_quantity,total_incident_duration,incident_year,count
0,0,36047040000,B4235,Bensonhurst East,Brooklyn,01/06/2013 07:05:52 PM,2,1035,2013,1
1,1,36047069602,B3183,Georgetown-Marine Park-Bergen Beach-Mill Basin,Brooklyn,01/13/2013 07:21:51 AM,3,1469,2013,1
2,2,36047052000,B2475,Flatbush,Brooklyn,07/30/2017 04:40:45 PM,2,481,2017,1
3,3,36047035300,B1012,Crown Heights North,Brooklyn,02/24/2020 10:11:19 AM,2,558,2020,1
4,4,36047007100,B0582,DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill,Brooklyn,12/19/2019 10:18:32 AM,2,1158,2019,1


In [215]:
df_gas_ = df_gas_.groupby(['incident_year', 'geoid']).agg({'engines_assigned_quantity': 'mean', 'total_incident_duration': 'mean', 'count': 'count'}).reset_index()

In [216]:
df_gas_.head()

,incident_year,geoid,engines_assigned_quantity,total_incident_duration,count
0,2013,36005000200,2.000000,1041.000000,2
1,2013,36005000400,2.200000,1351.000000,5
2,2013,36005001600,2.272727,1720.636364,11
3,2013,36005001900,2.000000,1170.000000,3
4,2013,36005002000,2.357143,1260.523810,42


In [217]:
df_gas_['incident_year'] = df_gas_['incident_year'].astype('int64')
df_acs['incident_year'] = df_acs['incident_year'].astype('int64')

In [218]:
merged_dfs = df_acs.merge(df_gas_, how='inner', on = ['incident_year','geoid'], validate="one_to_one")
merged_dfs = merged_dfs.sort_values(by = ['incident_year', 'geoid'])
merged_dfs.columns.to_list()

['Unnamed: 0',
 'total_housholds',
 'family_housholds',
 'nonfamily_housholds',
 'householder_living_alone',
 'householder_over65_living_alone',
 'grandparents_responsible_for_grandchildren',
 'avg_houshold_size',
 'nonfamily_housholds%',
 'lonely_housholder%',
 'lonely_housholder_over65%',
 'housholders_grandparents_responsible_for_grandchildren%',
 'not_us_citizen(abs, people)',
 'geoid',
 'unemployed',
 'unemployed%',
 'mean_houshold_income',
 'total_housing_units',
 'avg_year_built',
 'vacant_housing_units',
 'vacant_housing_units%',
 'total_population',
 'median_age',
 'not_us_citizen%',
 'incident_year',
 'engines_assigned_quantity',
 'total_incident_duration',
 'count']

In [219]:
merged_dfs['gas_leaks'] = merged_dfs['count']
merged_dfs['gas_leaks_per_person'] = merged_dfs['count']/merged_dfs['total_population']

merged_dfs.rename(columns={'engines_assigned_quantity':'avg_assigned_fdny_engines#',
                          'total_incident_duration':'avg_incident_duration',
                          'housholders_grandparents_responsible_for_grandchildren%':'%housh_grandp_resp_for_grandch'}, 
                  inplace = True)
columns = ['incident_year','geoid','gas_leaks','gas_leaks_per_person',
 'median_age','avg_houshold_size', 'avg_year_built',
 'total_population',         
 'total_housholds',
 'nonfamily_housholds',
 'householder_living_alone',
 'householder_over65_living_alone',
 'grandparents_responsible_for_grandchildren',
 'not_us_citizen(abs, people)',
 'unemployed',
  
 'nonfamily_housholds%',
 'lonely_housholder%',
 'lonely_housholder_over65%',
 '%housh_grandp_resp_for_grandch',
 'unemployed%',
 'mean_houshold_income',
 'total_housing_units',
 'vacant_housing_units%',
 'not_us_citizen%'
 ]

In [220]:
merged_dfs = merged_dfs[columns]
merged_dfs[merged_dfs < 0] = None
merged_dfs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11607 entries, 0 to 10045
Data columns (total 24 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   incident_year                               11607 non-null  int64  
 1   geoid                                       11607 non-null  int64  
 2   gas_leaks                                   11607 non-null  int64  
 3   gas_leaks_per_person                        11607 non-null  float64
 4   median_age                                  11590 non-null  float64
 5   avg_houshold_size                           11595 non-null  float64
 6   avg_year_built                              11607 non-null  float64
 7   total_population                            11607 non-null  int64  
 8   total_housholds                             11607 non-null  int64  
 9   nonfamily_housholds                         11607 non-null  int64  
 10  householde

In [221]:
merged_dfs_by_year = merged_dfs[['geoid','incident_year','gas_leaks','gas_leaks_per_person','median_age','avg_houshold_size', 'avg_year_built',
 'nonfamily_housholds%',
 'lonely_housholder%',
 'lonely_housholder_over65%',
 '%housh_grandp_resp_for_grandch',
 'unemployed%',
 'mean_houshold_income',
 'total_housing_units',
 'vacant_housing_units%',
 'not_us_citizen%']]

In [222]:
sorted_columns = merged_dfs_by_year.columns.to_list()
sorted_columns.sort()
merged_dfs_by_year = merged_dfs_by_year[sorted_columns]

In [223]:
merged_dfs_by_year.head()
columns = sorted(merged_dfs_by_year.columns)
merged_dfs_by_year=merged_dfs_by_year[columns]

In [224]:
merged_dfs_by_year.to_csv(base_path+ 'processed\important_(used_in_app)\Merged_asc_fdny_data.csv')

In [225]:
merged_dfs_by_year.head(5000)

,%housh_grandp_resp_for_grandch,avg_houshold_size,avg_year_built,gas_leaks,gas_leaks_per_person,geoid,incident_year,lonely_housholder%,lonely_housholder_over65%,mean_houshold_income,median_age,nonfamily_housholds%,not_us_citizen%,total_housing_units,unemployed%,vacant_housing_units%
0,9.576837,3.60,1953.839431,2,0.000412,36005000200,2013,18.040089,11.210097,72652.0,34.5,19.005197,11.198024,1476,8.5,8.7
1,4.164290,3.04,1972.667364,5,0.000936,36005000400,2013,13.519681,7.187678,80733.0,35.2,18.881917,7.821856,1912,5.7,8.3
2,1.748072,2.55,1969.786833,11,0.002109,36005001600,2013,33.470437,13.984576,44609.0,39.2,34.035990,15.819751,2043,6.6,4.8
3,0.000000,2.60,1966.332772,3,0.001171,36005001900,2013,35.499398,2.406739,42285.0,30.5,44.645006,24.863388,891,18.5,6.7
4,6.241611,2.84,1953.933724,42,0.004967,36005002000,2013,28.624161,12.953020,34044.0,35.9,29.798658,9.425260,3161,13.5,5.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5084,3.930530,2.66,1945.674785,21,0.003450,36061021303,2015,37.934186,6.444241,48354.0,35.9,52.376600,24.511253,2443,4.1,10.4
5085,1.585624,1.59,1958.549852,39,0.012970,36061021400,2015,70.137421,20.031712,47293.0,45.2,76.744186,10.375790,2026,11.0,6.6
5086,2.182092,2.80,1972.022276,41,0.010916,36061021500,2015,33.032355,10.007524,46489.0,29.2,42.136945,16.959531,1459,9.6,8.9
5087,1.465749,2.44,1944.288650,23,0.002734,36061021600,2015,41.160634,9.991026,85416.0,36.4,56.506132,9.211934,3771,8.2,11.3


In [226]:
df_gas_ = df_gas
df_gas_['incident_year'] = df_gas['incident_date_time'].str[6:10]
df_gas_['incident_month'] = df_gas['incident_date_time'].str[:2]
df_gas_['count'] = 1
df_gas_.head()

,Unnamed: 0,geoid,fire_box,ntaname,boroname,incident_date_time,engines_assigned_quantity,total_incident_duration,incident_year,count,incident_month
0,0,36047040000,B4235,Bensonhurst East,Brooklyn,01/06/2013 07:05:52 PM,2,1035,2013,1,01
1,1,36047069602,B3183,Georgetown-Marine Park-Bergen Beach-Mill Basin,Brooklyn,01/13/2013 07:21:51 AM,3,1469,2013,1,01
2,2,36047052000,B2475,Flatbush,Brooklyn,07/30/2017 04:40:45 PM,2,481,2017,1,07
3,3,36047035300,B1012,Crown Heights North,Brooklyn,02/24/2020 10:11:19 AM,2,558,2020,1,02
4,4,36047007100,B0582,DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill,Brooklyn,12/19/2019 10:18:32 AM,2,1158,2019,1,12


In [227]:
df_gas_ = df_gas_.groupby(['incident_year', 'incident_month','geoid']).agg({'engines_assigned_quantity': 'mean', 'total_incident_duration': 'mean', 'count': 'count'}).reset_index()

In [228]:
df_gas_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79776 entries, 0 to 79775
Data columns (total 6 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   incident_year              79776 non-null  object 
 1   incident_month             79776 non-null  object 
 2   geoid                      79776 non-null  int64  
 3   engines_assigned_quantity  79776 non-null  float64
 4   total_incident_duration    79776 non-null  float64
 5   count                      79776 non-null  int64  
dtypes: float64(2), int64(2), object(2)
memory usage: 3.7+ MB


In [229]:
df_gas_['incident_year'] = df_gas_['incident_year'].astype('int64')
df_acs['incident_year'] = df_acs['incident_year'].astype('int64')
df_gas_['incident_month'] = df_gas_['incident_month'].astype('int64')

In [230]:
merged_dfs = df_gas_.merge(df_acs, how='left', on = ['incident_year','geoid'])
merged_dfs = merged_dfs.sort_values(by = ['incident_year', 'incident_month','geoid'])
merged_dfs.columns.to_list()

['incident_year',
 'incident_month',
 'geoid',
 'engines_assigned_quantity',
 'total_incident_duration',
 'count',
 'Unnamed: 0',
 'total_housholds',
 'family_housholds',
 'nonfamily_housholds',
 'householder_living_alone',
 'householder_over65_living_alone',
 'grandparents_responsible_for_grandchildren',
 'avg_houshold_size',
 'nonfamily_housholds%',
 'lonely_housholder%',
 'lonely_housholder_over65%',
 'housholders_grandparents_responsible_for_grandchildren%',
 'not_us_citizen(abs, people)',
 'unemployed',
 'unemployed%',
 'mean_houshold_income',
 'total_housing_units',
 'avg_year_built',
 'vacant_housing_units',
 'vacant_housing_units%',
 'total_population',
 'median_age',
 'not_us_citizen%']

In [231]:
merged_dfs.head(10)

,incident_year,incident_month,geoid,engines_assigned_quantity,total_incident_duration,count,Unnamed: 0,total_housholds,family_housholds,nonfamily_housholds,...,unemployed,unemployed%,mean_houshold_income,total_housing_units,avg_year_built,vacant_housing_units,vacant_housing_units%,total_population,median_age,not_us_citizen%
0,2013,1,36005002000,2.333333,1162.833333,6,92.0,2980.0,2092.0,888.0,...,408.0,13.5,34044.0,3161.0,1953.933724,181.0,5.7,8456.0,35.9,9.425260
1,2013,1,36005002400,2.000000,911.000000,1,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2013,1,36005002500,2.000000,1062.000000,1,94.0,1814.0,1223.0,591.0,...,227.0,12.1,26129.0,2063.0,1949.309985,249.0,12.1,5108.0,27.0,25.078309
3,2013,1,36005002701,2.000000,822.500000,2,95.0,1059.0,835.0,224.0,...,252.0,18.5,36719.0,1105.0,1950.578733,46.0,4.2,3172.0,32.7,17.559899
4,2013,1,36005002800,2.000000,6190.000000,1,97.0,2340.0,1425.0,915.0,...,282.0,9.0,58256.0,2427.0,1961.982489,87.0,3.6,5804.0,39.9,6.805651
5,2013,1,36005003500,2.500000,996.500000,2,100.0,1289.0,899.0,390.0,...,220.0,14.9,30035.0,1346.0,1943.961367,57.0,4.2,3776.0,28.7,17.743644
6,2013,1,36005004001,2.000000,1323.000000,1,104.0,490.0,327.0,163.0,...,69.0,9.8,66040.0,564.0,1948.276596,74.0,13.1,1470.0,39.9,14.761905
7,2013,1,36005004100,2.000000,1203.500000,4,105.0,2025.0,1425.0,600.0,...,392.0,16.3,30864.0,2189.0,1957.325491,164.0,7.5,6218.0,25.9,9.295593
8,2013,1,36005004200,2.000000,1560.600000,5,106.0,2702.0,1766.0,936.0,...,622.0,23.5,37429.0,2797.0,1963.461387,95.0,3.4,7429.0,34.5,4.536277
9,2013,1,36005004300,2.000000,757.000000,1,107.0,1809.0,1195.0,614.0,...,295.0,14.1,35021.0,2005.0,1960.547382,196.0,9.8,5334.0,29.5,12.635921


In [232]:
merged_dfs['gas_leaks_per_person'] = merged_dfs['count']/merged_dfs['total_population']
merged_dfs['gas_leaks'] = merged_dfs['count']

merged_dfs.rename(columns={'engines_assigned_quantity':'avg_assigned_fdny_engines#',
                          'total_incident_duration':'avg_incident_duration',
                          'housholders_grandparents_responsible_for_grandchildren%':'%housh_grandp_resp_for_grandch'}, 
                  inplace = True)
columns = ['incident_year','incident_month','geoid','gas_leaks','gas_leaks_per_person','total_population']

In [233]:
merged_dfs = merged_dfs[columns]
merged_dfs[merged_dfs < 0] = None
merged_dfs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 79776 entries, 0 to 79775
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   incident_year         79776 non-null  int64  
 1   incident_month        79776 non-null  int64  
 2   geoid                 79776 non-null  int64  
 3   gas_leaks             79776 non-null  int64  
 4   gas_leaks_per_person  63189 non-null  float64
 5   total_population      63189 non-null  float64
dtypes: float64(2), int64(4)
memory usage: 4.3 MB


In [234]:
merged_dfs = merged_dfs.dropna()
merged_dfs.to_csv(base_path+ 'processed\important_(used_in_app)\Merged_asc_fdny_data_months.csv')

<h2> Merging without grouping by year </h2>

In [235]:
import glob
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join

years = [str(i) for i in range(2010, 2019)]
path_from = 'C:\\Users\\mskac\\machineLearning\\GasLeakConEd\\data\\processed\\acs_data\\'
path_to = 'C:\\Users\\mskac\\machineLearning\\GasLeakConEd\\data\\processed\\merged_acs_data\\'
base_path = 'C:/Users/mskac/machineLearning/GasLeakConEd/data/'

files = ['acs_merged_data' + str(year) + '.csv' for year in range(2010, 2019)]

In [236]:
dfs_acs_original = [pd.read_csv(join(path_to, files[i])) for i in range(3,9)]
for i in range(len(dfs_acs_original)):
    dfs_acs_original[i]['incident_year'] = 2010 +i

In [237]:
# we will have all years information in one dataframe to merge it with gas info
df_acs = pd.concat(dfs_acs_original, ignore_index = True)
df_acs['geoid'] = df_acs['geoid'].astype('int64')

In [238]:
df_gas = pd.read_csv(base_path+ 'processed\FULL_fdny_2013_2020.csv')
df_gas['geoid'] = df_gas['geoid'].astype('int64')

In [239]:
df_gas_ = df_gas[df_gas['incident_date_time'].str[6:10] < '2019']
df_gas_['count'] = 1
df_gas_.head()

C:\Users\mskac\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Unnamed: 0,geoid,fire_box,ntaname,boroname,incident_date_time,engines_assigned_quantity,total_incident_duration,count
0,0,36047040000,B4235,Bensonhurst East,Brooklyn,01/06/2013 07:05:52 PM,2,1035,1
1,1,36047069602,B3183,Georgetown-Marine Park-Bergen Beach-Mill Basin,Brooklyn,01/13/2013 07:21:51 AM,3,1469,1
2,2,36047052000,B2475,Flatbush,Brooklyn,07/30/2017 04:40:45 PM,2,481,1
5,5,36047013000,B1107,Bay Ridge,Brooklyn,10/12/2016 05:40:37 PM,2,2808,1
7,7,36047082600,B1576,Erasmus,Brooklyn,03/14/2013 12:36:28 AM,2,818,1


In [240]:
df_gas_ = df_gas_.groupby(['geoid']).agg({'engines_assigned_quantity': 'mean', 'total_incident_duration': 'mean', 'count': 'count'}).reset_index()

In [241]:
df_gas_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2139 entries, 0 to 2138
Data columns (total 4 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   geoid                      2139 non-null   int64  
 1   engines_assigned_quantity  2139 non-null   float64
 2   total_incident_duration    2139 non-null   float64
 3   count                      2139 non-null   int64  
dtypes: float64(2), int64(2)
memory usage: 67.0 KB


In [242]:
df_acs.columns.to_list()

['Unnamed: 0',
 'total_housholds',
 'family_housholds',
 'nonfamily_housholds',
 'householder_living_alone',
 'householder_over65_living_alone',
 'grandparents_responsible_for_grandchildren',
 'avg_houshold_size',
 'nonfamily_housholds%',
 'lonely_housholder%',
 'lonely_housholder_over65%',
 'housholders_grandparents_responsible_for_grandchildren%',
 'not_us_citizen(abs, people)',
 'geoid',
 'unemployed',
 'unemployed%',
 'mean_houshold_income',
 'total_housing_units',
 'avg_year_built',
 'vacant_housing_units',
 'vacant_housing_units%',
 'total_population',
 'median_age',
 'not_us_citizen%',
 'incident_year']

In [243]:
df_acs_new = df_acs.groupby(['geoid']).agg({
     'total_housholds': 'sum',
     'nonfamily_housholds': 'sum',
     'householder_living_alone': 'sum',
     'householder_over65_living_alone': 'sum',
     'grandparents_responsible_for_grandchildren': 'sum',
     'avg_houshold_size':'mean',
     
     'not_us_citizen(abs, people)': 'sum',
     'unemployed': 'sum',
     'mean_houshold_income':'mean',
     'total_housing_units': 'sum',
     'avg_year_built':'mean',
     'vacant_housing_units': 'sum',
     'total_population': 'sum',
     'median_age':'mean'
}).reset_index()

In [67]:
#############
df_acs = df_acs.groupby(['geoid']).agg({'avg_houshold_size':'mean', 'nonfamily_housholds%':'mean',
       'lonely_housholder%':'mean', 'lonely_housholder_over65%':'mean',
       'housholders_grandparents_responsible_for_grandchildren%':'mean',
       'not_us_citizen(abs, people)':'mean', 'unemployed%':'mean','mean_houshold_income':'mean', 'total_housing_units':'mean',
       'avg_year_built':'mean', 'occupied_housing_units%':'mean', 'vacant_housing_units%':'mean',
       'total_population':'sum', 'median_age':'mean', 'not_us_citizen%':'mean'}).reset_index()

In [244]:
df_acs_new.columns.to_list()

['geoid',
 'total_housholds',
 'nonfamily_housholds',
 'householder_living_alone',
 'householder_over65_living_alone',
 'grandparents_responsible_for_grandchildren',
 'avg_houshold_size',
 'not_us_citizen(abs, people)',
 'unemployed',
 'mean_houshold_income',
 'total_housing_units',
 'avg_year_built',
 'vacant_housing_units',
 'total_population',
 'median_age']

In [245]:
df_acs_new['nonfamily_housholds%'] = df_acs_new['nonfamily_housholds']/df_acs_new['total_housholds']*100
df_acs_new['lonely_housholder%'] = df_acs_new['householder_living_alone']/df_acs_new['total_housholds']*100
df_acs_new['lonely_housholder_over65%'] = df_acs_new['householder_over65_living_alone']/df_acs_new['total_housholds']*100
df_acs_new['%housh_grandp_resp_for_grandch'] = df_acs_new['grandparents_responsible_for_grandchildren']/df_acs_new['total_housholds']*100
df_acs_new['unemployed%'] = df_acs_new['unemployed']/df_acs_new['total_population']*100
df_acs_new['vacant_housing_units%'] = df_acs_new['vacant_housing_units']/df_acs_new['total_housing_units']*100
df_acs_new['not_us_citizen%'] = df_acs_new['not_us_citizen(abs, people)']/df_acs_new['total_population']*100


In [246]:
merged_dfs = df_acs_new.merge(df_gas_, how='inner', on = ['geoid'], validate="one_to_one")
merged_dfs = merged_dfs.sort_values(by = ['geoid'])
merged_dfs.columns.to_list()

['geoid',
 'total_housholds',
 'nonfamily_housholds',
 'householder_living_alone',
 'householder_over65_living_alone',
 'grandparents_responsible_for_grandchildren',
 'avg_houshold_size',
 'not_us_citizen(abs, people)',
 'unemployed',
 'mean_houshold_income',
 'total_housing_units',
 'avg_year_built',
 'vacant_housing_units',
 'total_population',
 'median_age',
 'nonfamily_housholds%',
 'lonely_housholder%',
 'lonely_housholder_over65%',
 '%housh_grandp_resp_for_grandch',
 'unemployed%',
 'vacant_housing_units%',
 'not_us_citizen%',
 'engines_assigned_quantity',
 'total_incident_duration',
 'count']

In [247]:
merged_dfs['gas_leaks'] = merged_dfs['count']
merged_dfs['gas_leaks_per_person'] = merged_dfs['count']/merged_dfs['total_population']

merged_dfs.rename(columns={'engines_assigned_quantity':'avg_assigned_fdny_engines#',
                          'total_incident_duration':'avg_incident_duration',
                          'housholders_grandparents_responsible_for_grandchildren%': '%housh_grandp_resp_for_grandch'}, 
                  inplace = True)
columns = ['geoid','gas_leaks','gas_leaks_per_person',
 'median_age','avg_houshold_size', 'avg_year_built',
 'nonfamily_housholds%',
 'lonely_housholder%',
 'lonely_housholder_over65%',
 '%housh_grandp_resp_for_grandch',
 'unemployed%',
 'mean_houshold_income',
 'total_housing_units',
 'vacant_housing_units%',
 'not_us_citizen%',
 ]

In [248]:
merged_dfs = merged_dfs[columns]
merged_dfs[merged_dfs < 0] = None

columns = sorted(merged_dfs.columns)
merged_dfs=merged_dfs[columns]
merged_dfs.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2051 entries, 0 to 2050
Data columns (total 15 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   %housh_grandp_resp_for_grandch  2051 non-null   float64
 1   avg_houshold_size               2045 non-null   float64
 2   avg_year_built                  2051 non-null   float64
 3   gas_leaks                       2051 non-null   int64  
 4   gas_leaks_per_person            2051 non-null   float64
 5   geoid                           2051 non-null   int64  
 6   lonely_housholder%              2051 non-null   float64
 7   lonely_housholder_over65%       2051 non-null   float64
 8   mean_houshold_income            2039 non-null   float64
 9   median_age                      2041 non-null   float64
 10  nonfamily_housholds%            2051 non-null   float64
 11  not_us_citizen%                 2051 non-null   float64
 12  total_housing_units             20

In [249]:
merged_dfs.to_csv(base_path+ 'processed\important_(used_in_app)\Merged_asc_fdny_data_all_years.csv')